In [ ]:
%load_ext line_profiler

In [1]:
import pandas as pd

In [2]:
from tardis import run_tardis
sim = run_tardis('tardis_example.yml')

/home/jals/miniconda3/envs/tardis/lib/python3.6/importlib/_bootstrap.py:219: QAWarning: pyne.data is not yet QA compliant.
  return f(*args, **kwds)


[tardis.plasma.standard_plasmas][INFO   ]  Reading Atomic Data from kurucz_cd23_chianti_H_He.h5 (standard_plasmas.py:87)
[tardis.io.atom_data.util][INFO   ]  Atom Data kurucz_cd23_chianti_H_He.h5 not found in local path. Exists in TARDIS Data repo /home/jals/Downloads/tardis-data/kurucz_cd23_chianti_H_He.h5 (util.py:35)
[py.warnings         ][WARNING]  /home/jals/miniconda3/envs/tardis/lib/python3.6/site-packages/IPython/core/interactiveshell.py:3331: PerformanceWarning: indexing past lexsort depth may impact performance.
  exec(code_obj, self.user_global_ns, self.user_ns)
 (warnings.py:99)
[tardis.io.atom_data.base][INFO   ]  Read Atom Data with UUID=6f7b09e887a311e7a06b246e96350010 and MD5=864f1753714343c41f99cb065710cace. (base.py:187)
[tardis.io.atom_data.base][INFO   ]  Non provided atomic data: synpp_refs, photoionization_data (base.py:193)
[tardis.simulation.base][INFO   ]  Starting iteration 1/20 (base.py:325)
[tardis.simulation.base][INFO   ]  Luminosity emitted = 7.96915e+42 

[tardis.simulation.base][INFO   ]  t_inner 10123.177 K -- next t_inner 11180.783 K (base.py:436)
[tardis.simulation.base][INFO   ]  Starting iteration 10/20 (base.py:325)
[tardis.simulation.base][INFO   ]  Luminosity emitted = 1.28170e+43 erg / s Luminosity absorbed = 4.19511e+42 erg / s Luminosity requested = 1.05928e+43 erg / s (base.py:447)
[tardis.simulation.base][INFO   ]  Plasma stratification:
	              t_rad    next_t_rad         w    next_w
	Shell                                                
	0      10666.223660  11427.410445  0.445097  0.500788
	5      11000.288585  11612.349822  0.163424  0.193129
	10     10822.884326  11373.245169  0.101641  0.121622
	15     10384.181273  11047.955461  0.078432  0.090937

 (base.py:433)
[tardis.simulation.base][INFO   ]  t_inner 11180.783 K -- next t_inner 10164.432 K (base.py:436)
[tardis.simulation.base][INFO   ]  Starting iteration 11/20 (base.py:325)
[tardis.simulation.base][INFO   ]  Luminosity emitted = 8.79074e+42 erg / s Lum

[tardis.simulation.base][INFO   ]  t_inner 10314.623 K -- next t_inner 11009.102 K (base.py:436)
[tardis.simulation.base][INFO   ]  Starting iteration 20/20 (base.py:325)
[tardis.simulation.base][INFO   ]  Luminosity emitted = 1.21062e+43 erg / s Luminosity absorbed = 3.88405e+42 erg / s Luminosity requested = 1.05928e+43 erg / s (base.py:447)
[tardis.simulation.base][INFO   ]  Simulation finished in 20 iterations and took 74.40 s (base.py:380)


## Data exploration in old GUI code

In [ ]:
from astropy import units as u
from tardis.analysis import LastLineInteraction
from tardis.util.base import species_tuple_to_string, species_string_to_tuple

atom_data = sim.plasma.atomic_data.atom_data
lines_data = sim.plasma.lines.reset_index().set_index('line_id')
line_interaction_analysis = LastLineInteraction.from_model(sim)


def select_in_line_interaction_analysis(wavelength_start, wavelength_end, packet_filter_mode='packet_nu'):
    # LastLineInteraction class's wavelength parameter setters slice the entire lines_data each time
    # (so modify object in global scope and later in lineinfo class scope)

    # Better approach: do it locally but how can it work with current structure of LastLineInteraction?
    line_interaction_analysis.packet_filter_mode = 'packet_nu'
    line_interaction_analysis.wavelength_start = wavelength_start * u.angstrom
    line_interaction_analysis.wavelength_end = wavelength_end * u.angstrom

    # WARNING: Time Bottleneck is this function - optimize way of slicing lines_data as per wavelength
    # see update_last_interaction_filter function in analysis.py

### Species Abundances Table: Code Exploration

In [ ]:
select_in_line_interaction_analysis(5000, 10000)

In [ ]:
selected_species_group = line_interaction_analysis.last_line_in.groupby(
    ['atomic_number', 'ion_number'])
selected_species_symbols = [species_tuple_to_string(
    item) for item in selected_species_group.groups.keys()]
selected_species_symbols

In [ ]:
selected_species_group.size(), selected_species_group.wavelength.count()

In [ ]:
line_interaction_analysis.last_line_in.shape[0], line_interaction_analysis.last_line_in.wavelength.count(
)

In [ ]:
selected_species_abundances = selected_species_group.size(
)/line_interaction_analysis.last_line_in.shape[0]
selected_species_abundances

In [ ]:
selected_species_abundances.index = pd.Index(
    selected_species_symbols, name='Species')
selected_species_abundances.name = 'Fractional Abundance'
selected_species_abundances.sort_values(ascending=False).to_frame()

### Species Abundances Table: Complete function

In [ ]:
def species_abundances_data(wavelength_start, wavelength_end, packet_filter_mode='packet_nu'):
    select_in_line_interaction_analysis(
        wavelength_start, wavelength_end, packet_filter_mode)

    selected_species_group = line_interaction_analysis.last_line_in.groupby(
        ['atomic_number', 'ion_number'])
    selected_species_symbols = [species_tuple_to_string(
        item) for item in selected_species_group.groups.keys()]

    selected_species_abundances = selected_species_group.size(
    )/line_interaction_analysis.last_line_in.shape[0]
    selected_species_abundances.index = pd.Index(
        selected_species_symbols, name='Species')
    selected_species_abundances.name = 'Fractional Abundance'
    return selected_species_abundances.sort_values(ascending=False).to_frame()

In [ ]:
species_abundances_data(3000, 12000)

#### Benchmarking

In [ ]:
%lprun - f species_abundances_data species_abundances_data(5000, 10000)

In [ ]:
%lprun - f select_in_line_interaction_analysis select_in_line_interaction_analysis(5000, 10000)

### Line count table: Code exploration

In [ ]:
from tardis.util.base import species_string_to_tuple

selected_species = 'Si II'
selected_species_tuple = species_string_to_tuple(selected_species)

In [ ]:
current_last_line_in = line_interaction_analysis.last_line_in.xs(
    key=(selected_species_tuple[0], selected_species_tuple[1]),
    level=['atomic_number', 'ion_number'],
    drop_level=False).reset_index()
current_last_line_in

In [ ]:
current_last_line_out = line_interaction_analysis.last_line_out.xs(
    key=(selected_species_tuple[0], selected_species_tuple[1]),
    level=['atomic_number', 'ion_number'],
    drop_level=False).reset_index()

current_last_line_out

In [ ]:
current_last_line_in.line_id.unique()

In [ ]:
current_last_line_out.line_id.unique()

In [ ]:
current_last_line_in['line_id_out'] = current_last_line_out.line_id
# Q: why, are they really corresponding?

In [ ]:
last_line_in_string = []
last_line_count = []
grouped_line_interactions = current_last_line_in.groupby(
    ['line_id', 'line_id_out'])
exc_deexc_string = 'exc. %d-%d (%.2f A) de-exc. %d-%d (%.2f A)'

In [ ]:
# all(grouped_line_interactions.wavelength.count() == grouped_line_interactions.size())

In [ ]:
grouped_line_interactions.size()

In [ ]:
lines_data = lines_data.reset_index().set_index('line_id')
lines_data

In [ ]:
for line_id, count in grouped_line_interactions.size().iteritems():
    current_line_in = lines_data.loc[line_id[0]]
    current_line_out = lines_data.loc[line_id[1]]
    last_line_in_string.append(exc_deexc_string % (
        current_line_in['level_number_lower'],
        current_line_in['level_number_upper'],
        current_line_in['wavelength'],
        current_line_out['level_number_upper'],a
        current_line_out['level_number_lower'],
        current_line_out['wavelength']))
    last_line_count.append(count)

In [ ]:
last_line_in_string

In [ ]:
line_counts = pd.DataFrame(last_line_count, index=last_line_in_string)
line_counts.columns = ['Num. pkts {}'.format(len(current_last_line_in))]
line_counts

### Line count table: Complete function

In [ ]:
def last_line_counts(selected_species):
    selected_species_tuple = species_string_to_tuple(selected_species)

    current_last_line_in = line_interaction_analysis.last_line_in.xs(
        key=(selected_species_tuple[0], selected_species_tuple[1]),
        level=['atomic_number', 'ion_number'],
        drop_level=False).reset_index()

    current_last_line_out = line_interaction_analysis.last_line_out.xs(
        key=(selected_species_tuple[0], selected_species_tuple[1]),
        level=['atomic_number', 'ion_number'],
        drop_level=False).reset_index()

    current_last_line_in['line_id_out'] = current_last_line_out.line_id

    last_line_in_string = []
    last_line_count = []
    grouped_line_interactions = current_last_line_in.groupby(
        ['line_id', 'line_id_out'])
    exc_deexc_string = 'exc. %d-%d (%.2f A) de-exc. %d-%d (%.2f A)'

    for line_id, count in grouped_line_interactions.size().iteritems():
        current_line_in = lines_data.loc[line_id[0]]
        current_line_out = lines_data.loc[line_id[1]]
        last_line_in_string.append(exc_deexc_string % (
            current_line_in['level_number_lower'],
            current_line_in['level_number_upper'],
            current_line_in['wavelength'],
            current_line_out['level_number_upper'],
            current_line_out['level_number_lower'],
            current_line_out['wavelength']))
        last_line_count.append(count)

    line_counts = pd.DataFrame(last_line_count, index=last_line_in_string)
    line_counts.columns = ['Num. pkts {}'.format(len(current_last_line_in))]
    return line_counts

In [ ]:
last_line_counts('Si II')

## Line info widget demo using slider

### Creating component widgets

In [ ]:
import ipywidgets as ipw
initial_range = [5000, 10000]

wavelength_slider = ipw.IntRangeSlider(
    value=initial_range,
    min=0,
    max=30000,  # what is ideal - infinity or something else?
    step=1,
    description='Wavelength:',
    disabled=False,
    continuous_update=False,
    orientation='horizontal',
    readout=True,
    readout_format='d',
)
wavelength_slider.layout.width = '350px'

# wavelength_slider

In [ ]:
import qgrid
species_abundances_table = qgrid.show_grid(species_abundances_data(*initial_range))
species_abundances_table.layout.width = '450px'
# species_abundances_table

In [ ]:
line_counts_table = qgrid.show_grid(last_line_counts(species_abundances_table.df.index[0]))
line_counts_table.layout.width = '450px'
# line_counts_table

### Attaching event listeners

In [ ]:
def update_species_abundances(change):
    # Update data in species abundance table
    species_abundances_table.df = species_abundances_data(*change['new'])
    
    # Get index of 0th row in species abundance table
    species0 = species_abundances_table.df.index[0]
    
    # Also update line counts table by triggering its event listener
    # Listener won't trigger if last row selected in species abundance table was also 0th
    if species_abundances_table.get_selected_rows() == [0]:
        species_abundances_table.change_selection([])  # Unselect rows
    # Select 0th row in count table which will trigger update_last_line_counts
    species_abundances_table.change_selection([species0])

wavelength_slider.observe(update_species_abundances, names='value')

In [ ]:
def update_last_line_counts(event, qgrid_widget):
    # Don't execute function if no row was selected implicitly (by api)
    if event["new"] == [] and event["source"] == "api":
        return
        
    # Get species from selected row in species abundance table
    species_selected = species_abundances_table.df.index[event["new"][0]]
    
    # Update data in line counts table
    line_counts_table.df = last_line_counts(species_selected)
    
species_abundances_table.on("selection_changed", update_last_line_counts)

### Laying out all components together

In [ ]:
ipw.VBox([wavelength_slider,
          ipw.Box([species_abundances_table, line_counts_table], 
                  layout=dict(display="flex", align_items="flex-start", justify_content="space-between")
                 )
         ])

### Testing other ways to interact with df

In [ ]:
out = ipw.Output(layout={'border': '1px solid yellow'})
out

In [ ]:
species_abundances_data(*(1000, 2000))

In [ ]:
def update_line_info(wavelength_slider):
    #     return line_info(sim, *wavelength_slider.value)
    return wavelength_slider


ipw.interact(update_line_info, wavelength_slider)

## Listening on Plotly graph interaction

In [3]:
wavelength = sim.runner.spectrum.wavelength
luminosity = sim.runner.spectrum.luminosity_density_lambda
wavelength, luminosity

(<Quantity [20000.        , 19922.30301823, 19845.20738242, ...,
              500.14629279,   500.09751902,   500.04875475] Angstrom>,
 <Quantity [1.44619994e+37, 2.51270471e+37, 1.27721039e+37, ...,
            0.00000000e+00, 0.00000000e+00, 0.00000000e+00] erg Hz / Angstrom>)

In [4]:
import plotly.graph_objects as go
import ipywidgets as ipw

In [79]:
import time

### Interaction on BoxSelect on line (+hidden markers) plot

In [87]:
fig = go.FigureWidget([
    go.Scatter(x=wavelength, y=luminosity,
#                mode="lines+markers", marker=dict(opacity=0) # Hides markers - time consuming in rendering!
               ),
    
    # Hide a one point scatter trace, to bring boxselect in modebar
    go.Scatter(x=wavelength[0], y=luminosity[0], mode='markers',
               marker=dict(opacity=0), showlegend=False
               )
],
    layout=go.Layout(
    xaxis=dict(title='Wavelength',
               exponentformat='none',
#                rangeslider=dict(visible=True), range=[3000, 9000]
              ),
    yaxis=dict(title='Luminosity', exponentformat='e',
#                fixedrange=False
              ),
    dragmode='select',
    selectdirection='h'),
)

In [88]:
from plotly.callbacks import BoxSelector
spec = fig.data[0]
out = ipw.Output(layout={'border': '1px solid black'})

@out.capture(clear_output=True, wait=True)
def selection_handler(trace, points, selector):
    if isinstance(selector, BoxSelector):
    #     t=time.time()
        fig.layout.shapes=[dict(type="rect",
                xref="x",
                yref="y",
                x0=selector.xrange[0],
                y0=selector.yrange[0],
                x1=selector.xrange[1],
                y1=selector.yrange[1],
                line=dict(
                    color="Salmon",
                    width=1,
                ),
                fillcolor="LightPink",
                opacity=0.5
            )]
        print(selector.xrange,
    #           time.time()-t
             )
        
spec.on_selection(selection_handler)

In [89]:
display(fig, out)

FigureWidget({
    'data': [{'type': 'scatter',
              'uid': '9129948f-c6e1-4ae2-a23e-d2ad14eb3e29',
 …

Output(layout=Layout(border='1px solid black'))

#### Status
- Now works faster, since scatter plot is reduced to one point
- So far is the best way line info selection can be made in plotly


- Interaction feels smoother (to me) than old gui way of highlighting edge and dragging, there was also no free-way of moving the selection rectangle without stretching to desired position and then resizing.


- Additional enhancements ***(if required?)***:

Feature to Add | Bottleneck
--- | ----
Scroll Zoom | controlled by [config options](https://plotly.com/python/configuration-options/) which only works when using renderers framework [mode of display](https://plotly.com/python/renderers/#displaying-figures), but we want figurewidget mode to show them together with ipywidgets
Permamently show Modebar (Alternative: Show a custom toggle button for select wavelength range) | Same as above

### Interactions on range slider

In [71]:
fig2 = go.FigureWidget([go.Scatter(x=wavelength, y=luminosity, mode="lines",
                                 )],
                      layout=go.Layout(
                          xaxis=dict(title='Wavelength', exponentformat='none',
                                     range=[3000, 9000], 
                                    rangeslider=dict(visible=True)),
                          yaxis=dict(title='Luminosity', exponentformat='e',
                                    fixedrange=False # To re-enable zoom along Y-axis
                                    ),
                      ))

In [72]:
fig2

FigureWidget({
    'data': [{'mode': 'lines',
              'type': 'scatter',
              'uid': 'a1e4b378-…

In [83]:
out2 = ipw.Output(layout={'border': '1px solid black'})
out2

Output(layout=Layout(border='1px solid black'))

#### Listening range

In [80]:
fig2.layout.xaxis.range

(3649.7028756031086, 7634.680182648806)

In [ ]:
# Try to listen changes on `fig2.layout.xaxis.range`?
# @out2.capture(clear_output=True, wait=True)
# def handler(change):
#     print(change)

# fig2.observe(handler, names)

In [82]:
# Listen relayout event triggered
@out2.capture(clear_output=True, wait=True)
def handler(change):
#   print({k: change[k] for k in ['name', 'old', 'new']})
#   if change['new'] and hasattr(change['new']['relayout_data'], 'xaxis.range'):
    print(change['new']['relayout_data']['xaxis.range'])

fig2.observe(handler, 
             names='_js2py_relayout'
            )

In [78]:
fig2.unobserve_all()

> Slider widths keep changing, need to listen on mouse up 
- the time counter approach as in JS solution on SO (retrying)
- Use a button to listen (like in old gui)

#### Status
- Now, zooming on Y-axis works


- Bottleneck: The selection in range slider always zoom but in actual we just want a simple selection area to be drawn which shows the region selected among the entire spectra


- Combining with previous plot (**if required?**) - this will aid in zooming since spectrums are really big horizontally and it makes it easier *to focus on a range* (Demo above by uncommenting). Besides, since we can add [range selector buttons](https://plotly.com/python/range-slider/#basic-range-slider-and-range-selectors) on it, this can allow us to divide pre-set ranges for visible, UV, infra EM bands in which we want spectrum to study (if such a use case is there).

### Interaction on drawing rectangle

In [105]:
fig5 = go.Figure(go.Scatter(x=[1,2,3], y=[1,3,6]))
fig5.add_shape(
    editable=True,
    x0=1, x1=3, y0=0, y1=4,
    xref='x', yref='y'
)
fig5.update_layout(dragmode='drawrect')
shp = fig5.layout.shapes[0]

In [106]:
fig5.show(config={'modeBarButtonsToAdd':[
                                        'drawrect',
                                        'eraseshape'
                                       ]})

In [107]:
# out5.unobserve_all()
out5 = ipw.Output(layout={'border': '1px solid black'})
out5

Output(layout=Layout(border='1px solid black'))

In [100]:
@out5.capture()
def shape_handler(shp_obj, x0, x1):
    print(x0, x1)

shp.on_change(shape_handler, 'x0', 'x1', append=True)

In [108]:
# @out5.capture()
# def handler(change):
#     print(change)
    
# fig5.observe(handler)

In [102]:
# fig5.layout.shapes=None

In [110]:
fig5.layout.shapes[0].x1 = 4

In [111]:
fig5.layout.activeshape

layout.Activeshape()

#### Status
- Looks helpful but doesn't fulfill our needs in line info gui:

Problem | Cause
--- | ---
Unable to listen Callbacks on modifying shape (I opened an issue) | The user interactions with a drawn shape are exposed by `layout.shape` is not updating on editing/adding new shapes
Need to display drawrect option in modebar | Modear config can't be controlled with figure widgets as explained above

### Misc

In [38]:
fig = go.Figure() # or any Plotly Express function e.g. px.bar(...)
fig.add_trace(spec)

import dash
import dash_core_components as dcc
import dash_html_components as html

app = dash.Dash()
app.layout = html.Div([
    dcc.Graph(figure=fig)
])

app.run_server(debug=True, use_reloader=True)  # Turn off reloader if inside Jupyter

Dash is running on http://x86_64-conda_cos6-linux-gnu:8050/

Dash is running on http://x86_64-conda_cos6-linux-gnu:8050/

 in production, use a production WSGI server like gunicorn instead.

 in production, use a production WSGI server like gunicorn instead.

 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: on


gaierror: [Errno -2] Name or service not known

In [ ]:
import plotly.graph_objects as go

import numpy as np
np.random.seed(1)

x = np.random.rand(100)
y = np.random.rand(100)

f = go.FigureWidget([go.Scatter(x=x, y=y, mode='markers')])

scatter = f.data[0]
colors = ['#a3a7e4'] * 100
scatter.marker.color = colors
scatter.marker.size = [10] * 100
f.layout.hovermode = 'closest'


# create our callback function
def update_point(trace, points, selector):
#     c = list(scatter.marker.color)
#     s = list(scatter.marker.size)
#     for i in points.point_inds:
#         c[i] = '#bae2be'
#         s[i] = 20
#         with f.batch_update():
#             scatter.marker.color = c
#             scatter.marker.size = s
    with out:
        print('heya')
        print(trace, points, selector, sep='\n')
        print('hoha')

scatter.on_selection(update_point)

display(f, out)

In [26]:
!export QT_API=pyqt

In [27]:
from tardis.gui import interface
interface.show(sim)

ImportError: QT_API was not set! Please exit the IPython console
 and at the bash prompt use : 

 export QT_API=pyside 
 or
 export QT_API=pyqt 

 For more information refer to user guide.

### Make config setting work with widgets
the show method return None that's why it fails - see a better way

In [112]:
ipw.VBox([fig2.show(config=config), out])

NameError: name 'config' is not defined